# Verification of the sequential and parallel programming of turbulence models

In [ ]:
import os
from trustutils import run

Comparison of sequential and parallel computation for the following turbulence model:
- $k-\varepsilon$ standard in VDF
- $k-\varepsilon$ standard in VEF
- $k-\varepsilon$ realizable in VDF
- $k-\varepsilon$ realizable in VEF
- $k-\omega$ standard in VEF
- $k-\omega$ SST in VEF

We perform 2 iterations in sequential and 2 in parallel before using `compare_lata`. The test case is a 5x5 nodes square with two walls. We export the velocity, the pressure, the $y^+$, the turbulent viscosity, the kinetic energy and the chosen dissipation scale.

## Launch the computation

In [ ]:
list_cases = [
    "SEQ_VDF_KEPS_STD",
    "SEQ_VDF_KEPS_REA",
    "SEQ_VEF_KEPS_STD",
    "SEQ_VEF_KEPS_REA",
    "SEQ_VEF_KOMEGA_STD",
    "SEQ_VEF_KOMEGA_SST",
]

In [ ]:
run.reset() 

#
run.initBuildDirectory()
builddir = run.BUILD_DIRECTORY
os.chdir(builddir)

# SEQ cases
for seqcase in list_cases:
    run.addCase(seqcase, "test.data")

# PAR cases
for seqcase in list_cases:
    parcase = seqcase.replace("SEQ", "PAR")
    therun = run.addCaseFromTemplate(f"{seqcase}/test.data",
                                     targetDirectory=f"{parcase}",
                                     targetData=f"test.data",
                                     dic={},
                                     nbProcs=2)
    therun.partition()

#
run.printCases()
run.runCases()

## Performance Chart

In [ ]:
run.tablePerf()

## Compare the SEQ/PAR latas

In [ ]:
os.chdir(run.BUILD_DIRECTORY)
for seqcase in list_cases:
    cmd = f"compare_lata {seqcase}/test.lata {seqcase.replace('SEQ', 'PAR')}/PAR_test.lata --max_delta"
    !{cmd}